## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-10-11-41-06 +0000,bbc,Musk says X outcry is 'excuse for censorship',https://www.bbc.com/news/articles/ce3kqzepp5zo...
1,2026-01-10-11-32-44 +0000,bbc,Rescuers race to find dozens missing after dea...,https://www.bbc.com/news/articles/cn42jqlqy4xo...
2,2026-01-10-11-21-43 +0000,nyt,Acquired Tastes,https://www.nytimes.com/2026/01/10/briefing/ac...
3,2026-01-10-11-19-13 +0000,nyt,"Trump Eyes Greenland, and Europe Figures Its B...",https://www.nytimes.com/2026/01/10/world/europ...
4,2026-01-10-11-11-38 +0000,bbc,Video emerges of Minneapolis shooting filmed b...,https://www.bbc.com/news/articles/cz7yv4524gqo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,62
28,ice,22
113,iran,20
267,oil,18
84,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
117,2026-01-10-00-23-02 +0000,nypost,President Trump wears ‘Happy Trump’ lapel pin ...,https://nypost.com/2026/01/09/us-news/presiden...,156
112,2026-01-10-00-36-00 +0000,wsj,Iran’s Supreme Leader Says He Won’t Bow Down t...,https://www.wsj.com/world/middle-east/irans-su...,119
310,2026-01-09-12-23-44 +0000,nyt,Trump Indicates He Will Meet Venezuela’s Macha...,https://www.nytimes.com/2026/01/09/world/ameri...,116
144,2026-01-09-22-58-00 +0000,wsj,Trump called Republican senators to personally...,https://www.wsj.com/politics/policy/angry-trum...,112
92,2026-01-10-01-40-34 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
117,156,2026-01-10-00-23-02 +0000,nypost,President Trump wears ‘Happy Trump’ lapel pin ...,https://nypost.com/2026/01/09/us-news/presiden...
94,111,2026-01-10-01-35-55 +0000,nypost,Anti-ICE lefties insist Renee Nicole Good was ...,https://nypost.com/2026/01/09/us-news/lefties-...
122,58,2026-01-10-00-05-41 +0000,bbc,South Africa's strained ties with US face new ...,https://www.bbc.com/news/articles/c62wxezynk2o...
310,54,2026-01-09-12-23-44 +0000,nyt,Trump Indicates He Will Meet Venezuela’s Macha...,https://www.nytimes.com/2026/01/09/world/ameri...
84,50,2026-01-10-02-00-00 +0000,wsj,America’s unemployment rate is low. But so is ...,https://www.wsj.com/economy/jobs/job-market-co...
188,44,2026-01-09-20-52-14 +0000,nypost,Billions in healthcare fraud discovered in Cal...,https://nypost.com/2026/01/09/us-news/billions...
244,40,2026-01-09-17-42-06 +0000,bbc,Owner of Swiss ski bar held in custody after d...,https://www.bbc.com/news/articles/c4g0mq520zeo...
33,39,2026-01-10-09-56-25 +0000,nypost,Texas judge threatens to have Amazon workers f...,https://nypost.com/2026/01/10/us-news/texas-ju...
308,32,2026-01-09-12-40-09 +0000,cbc,Iran demonstrations against regime continue as...,https://www.cbc.ca/news/world/iran-protests-fr...
256,32,2026-01-09-17-04-58 +0000,nypost,Mark Zuckerberg’s Meta makes massive bet on nu...,https://nypost.com/2026/01/09/business/mark-zu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
